## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [7]:
def blight_model():
    import pandas as pd
    import numpy as np
    import time
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier,  GradientBoostingClassifier
    from sklearn.model_selection import cross_val_score, KFold
    from sklearn.model_selection import StratifiedKFold
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.metrics import classification_report
    from sklearn.metrics import roc_auc_score
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.cluster import KMeans
    #Load data
    train = pd.read_csv(r"train.csv", encoding = "cp1252",index_col = 'ticket_id')
    test = pd.read_csv(r"test.csv", encoding = "cp1252",index_col = 'ticket_id')
    addresses = pd.read_csv(r"addresses.csv", encoding = "cp1252",index_col = 'ticket_id')
    latlons = pd.read_csv(r"latlons.csv", encoding = "cp1252")
    #     print('train: {},\ntest:  {}'.format(train.shape,test.shape))
    #     return train,test,addresses,latlons    
    # train,test,addresses,latlons = read_data()
               # train.head(2)
        
    #Combine daatasets
    train = train.merge(addresses, how = 'left',left_index = True, right_index = True ).merge(latlons, how = 'left', on = 'address')
    test = test.merge(addresses, how = 'left',left_index = True, right_index = True ).reset_index().merge(latlons, how = 'left', on = 'address').set_index('ticket_id')
    # #Transform lat and lon to a three dimensional space
#     train['x'] = np.cos(train['lat']) * np.cos(train['lon'])
#     train['y'] = np.cos(train['lat']) * np.sin(train['lon'])
#     train['z'] = np.sin(train['lat']) 
#     test['x'] = np.cos(test['lat']) * np.cos(test['lon'])
#     test['y'] = np.cos(test['lat']) * np.sin(test['lon'])
#     test['z'] = np.sin(test['lat'])
    
    test = test.fillna(test.mean())
    # Needed colomns
    numerical_columns = ['late_fee','judgment_amount','lat','lon','delta_date','violation_street_number', 'fine_amount', 'admin_fee', 'discount_amount', 'clean_up_cost']    
    # possible to add state, zip_code
    cat_columns = ['state','agency_name','inspector_name','country','violation_code','disposition']
    train_columns = [i for i in train.columns if i in (numerical_columns+cat_columns)]
    target_columns = ['compliance']    

    #Evaluate date_diff columns train
    train.loc[:,'hearing_date'] = pd.to_datetime(train.loc[:,'hearing_date'], format='%Y-%m-%d')
    train.loc[:,'ticket_issued_date'] = pd.to_datetime(train.loc[:,'ticket_issued_date'], format='%Y-%m-%d')
    train['delta_date'] = train['hearing_date'] - train['ticket_issued_date']
    train['delta_date'] = train['delta_date'].dt.total_seconds()    
    #Evaluate date_diff columns test
    test.loc[:,'hearing_date'] = pd.to_datetime(test.loc[:,'hearing_date'], format='%Y-%m-%d')
    test.loc[:,'ticket_issued_date'] = pd.to_datetime(test.loc[:,'ticket_issued_date'], format='%Y-%m-%d')
    test['delta_date'] = test['hearing_date'] - test['ticket_issued_date']
    test['delta_date'] = test['delta_date'].dt.total_seconds()
    test['delta_date'] = test['delta_date'].fillna(0)
    #Test set for output
    test = test[(numerical_columns+cat_columns)]    
    #Drop not needed class
    train_without_null_target = train[train['compliance'].isnull()!=True]
    #     print(train_without_null_target.shape)    
    
    # #Shuffle Data
    train_without_null_shuffle = train_without_null_target.sample(frac=1, random_state = 0, axis = 0)
    # Select data
    train_without_null_shuffle = train_without_null_shuffle[(numerical_columns+cat_columns+target_columns)].dropna()
    
    # Train set for output
    X = train_without_null_shuffle[(numerical_columns+cat_columns)]
    #Transform lat lon data
    
    kmeans = KMeans(n_clusters = 15, init ='k-means++')
    X['cluster_label'] = kmeans.fit_predict(X[['lat','lon']])
    X['cluster_label'] = X['cluster_label'].astype('object')
    X = X.drop(['lat','lon'], axis=1)

    # #Deal with categorical data test
    test['cluster_label'] = kmeans.predict(test[['lat','lon']])
    test['cluster_label'] = test['cluster_label'].astype('object')
    X = pd.get_dummies(X)
    y = train_without_null_shuffle[target_columns]
    # #Deal with categorical data test
    test = pd.get_dummies(test)
    test = test.reindex(columns = X.columns, fill_value=0)
    
    # Model already have fitted with RandomSearch and best paramerts have selected because of productivity of course autograde online system     
#   clf = RandomForestClassifier(max_features= 'auto',
#                                  max_depth=35,
#                                  criterion='entropy',
#                                  class_weight =  None,
#                                  random_state=1001)
#   gs = RandomizedSearchCV(clf, param_distributions=grid, scoring='roc_auc', cv=cv,n_iter=20,
#                             n_jobs=-1,verbose=5, random_state=1001)
#   gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv)
#   gs.fit(X, y.values.ravel())  
#   gs.best_params_
    clf = GradientBoostingClassifier(random_state=240, warm_start=True,
                                learning_rate=0.01, max_depth=5)
    clf.fit(X, y.values.ravel())   
    return pd.Series(clf.predict_proba(test)[:,1], dtype='float64',index = test.index)
# blight_model()
# https://www.coursera.org/learn/python-machine-learning/discussions/weeks/4/threads/Gs4NE2CeEeeqKgpTZZjjFg